In [16]:
import sqlite3 as sq
from bs4 import BeautifulSoup
import pandas as pd
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()


In [17]:
tickers_csv = pd.read_excel('..\src\Copy of NYSE_Stocks.xlsx')
tickers_list= tickers_csv['Symbol'].tolist()
ticker_count = len(tickers_list)
print(f"Number of unique tickers: {ticker_count}")

Task exception was never retrieved
future: <Task finished name='Task-18142' coro=<fetch_all() done, defined at C:\Users\rodri\AppData\Local\Temp\ipykernel_16400\1157234605.py:7> exception=TypeError("fetch() missing 1 required positional argument: 'ticker'")>
Traceback (most recent call last):
  File "C:\Python311\Lib\asyncio\tasks.py", line 267, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\Users\rodri\AppData\Local\Temp\ipykernel_16400\1157234605.py", line 13, in fetch_all
    tasks.append(fetch(session, url, headers, pbar))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: fetch() missing 1 required positional argument: 'ticker'
Fetching data:   0%|                                                       | 0/8970 [10:15<?, ?it/s]
Task exception was never retrieved
future: <Task finished name='Task-18141' coro=<fetch_all() done, defined at C:\Users\rodri\AppData\Local\Temp\ipykernel_16400\2286219434.py:7> exception=TypeError("fetch() missing

Number of unique tickers: 8970


In [27]:
# ... [resto do código anterior]

async def fetch(session, url, headers, ticker):  # Adicionado argumento ticker
    async with session.get(url, headers=headers) as response:
        data = await response.text()
        
        # Verificar o conteúdo para apenas alguns tickers
        if ticker in ["AAPL", "MSFT", "GOOGL"]:
            print(f"Content for {ticker}: {data[:1000]}")  # Imprime os primeiros 1000 caracteres
        
        return data

async def fetch_all(ticker_list, headers):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, ticker in enumerate(ticker_list, 1):
            url = f'https://finance.yahoo.com/quote/{ticker}?p={ticker}'
            tasks.append(fetch(session, url, headers, ticker))  # Corrigido para passar o ticker
            await asyncio.sleep(2)  # Aumentado o delay para 2 segundos
            print(f"Processed {idx}/{len(ticker_list)}: {ticker}")  # Imprime o progresso
        return await asyncio.gather(*tasks)

# ... [resto do código]

def extract_main_data(html_content, ticker):
    print(html_content[:1000])
    try:
        soup = BeautifulSoup(html_content, 'lxml')
        
        # Extract Market Cap
        market_cap_tag = soup.find("td", {"data-test": "MARKET_CAP-value"})
        market_cap = market_cap_tag.text if market_cap_tag else "N/A"
        
        # Extract Earnings Date
        earnings_date_tag = soup.find("td", {"data-test": "EARNINGS_DATE-value"})
        if earnings_date_tag and "-" in earnings_date_tag.text:
            earnings_date_min, earnings_date_max = map(str.strip, earnings_date_tag.text.split("-"))
        elif earnings_date_tag:
            earnings_date_min, earnings_date_max = earnings_date_tag.text, earnings_date_tag.text
        else:
            earnings_date_min, earnings_date_max = "N/A", "N/A"
        
        return {
            'Ticker': ticker,
            'Market Cap': market_cap,
            'Earnings Date Min': earnings_date_min,
            'Earnings Date Max': earnings_date_max
        }
    except Exception as e:
        print(f"Error extracting data for {ticker}. Error: {e}")
        return {
            'Ticker': ticker,
            'Market Cap': 'N/A',
            'Earnings Date Min': 'N/A',
            'Earnings Date Max': 'N/A'
        }

def get_main_data_for_tickers(ticker_list): 
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    loop = asyncio.get_event_loop()
    html_contents = loop.run_until_complete(fetch_all(ticker_list, headers))
    
    data_list = [extract_main_data(html, ticker) for html, ticker in zip(html_contents, ticker_list)]
    
    return pd.DataFrame(data_list)

In [29]:
df = get_main_data_for_tickers(tickers_list[0:20])
df

Processed 1/20: A
Processed 2/20: AA
Processed 3/20: AAA
Processed 4/20: AAAU
Processed 5/20: AAC
Processed 6/20: AACG
Processed 7/20: AACI
Processed 8/20: AACT
Processed 9/20: AADI
Processed 10/20: AADR
Processed 11/20: AAIC
Processed 12/20: AAL
Processed 13/20: AAMC
Processed 14/20: AAME
Processed 15/20: AAN
Processed 16/20: AAOI
Processed 17/20: AAON
Processed 18/20: AAP
Processed 19/20: AAPB
Processed 20/20: AAPD
<!DOCTYPE html><html data-color-theme="light" id="atomic" class="NoJs chrome desktop" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charSet="utf-8"/><title>Agilent Technologies, Inc. (A) Stock Price, News, Quote &amp; History - Yahoo Finance</title><meta name="keywords" content="A, Agilent Technologies, Inc., A stock chart, Agilent Technologies, Inc. stock chart, stock chart, stocks, quotes, finance"/><meta http-equiv="x-dns-prefetch-control" content="on"/><met

Task exception was never retrieved
future: <Task finished name='Task-18143' coro=<fetch_all() done, defined at C:\Users\rodri\AppData\Local\Temp\ipykernel_16400\3110836931.py:14> exception=TypeError('fetch() takes 4 positional arguments but 5 were given')>
Traceback (most recent call last):
  File "C:\Python311\Lib\asyncio\tasks.py", line 267, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "C:\Users\rodri\AppData\Local\Temp\ipykernel_16400\3110836931.py", line 20, in fetch_all
    tasks.append(fetch(session, url, headers, pbar, ticker))  # Passando ticker para fetch
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: fetch() takes 4 positional arguments but 5 were given
Fetching data:   0%|                                                       | 0/8970 [10:06<?, ?it/s]

<!DOCTYPE html>
  <html lang="en-us"><head>
  <meta http-equiv="content-type" content="text/html; charset=UTF-8">
      <meta charset="utf-8">
      <title>Yahoo</title>
      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">
      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
      <style>
  html {
      height: 100%;
  }
  body {
      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;
      background-size: cover;
      height: 100%;
      text-align: center;
      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;
  }
  table {
      height: 100%;
      width: 100%;
      table-layout: fixed;
      border-collapse: collapse;
      border-spacing: 0;
      border: none;
  }
  h1 {
      font-size: 42px;
      font-weight: 400;
      color: #400090;
  }
  p {
      color: #1A1A1A;
  }
  #message-1 {
      font-weight: bold;
      margin: 0;
  }
  #message-2 {
      display: 


c:\Users\rodri\Apps\venvs\.trading\Lib\site-packages\bs4\builder\_lxml.py:90: RuntimeWarning: coroutine 'fetch' was never awaited
  for key, value in list(mapping.items()):


<!DOCTYPE html><html data-color-theme="light" id="atomic" class="NoJs chrome desktop" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charSet="utf-8"/><title>Altisource Asset Management Corporation (AAMC) Stock Price, News, Quote &amp; History - Yahoo Finance</title><meta name="keywords" content="AAMC, Altisource Asset Management Corporation, AAMC stock chart, Altisource Asset Management Corporation stock chart, stock chart, stocks, quotes, finance"/><meta http-equiv="x-dns-prefetch-control" content="on"/><meta property="twitter:dnt" content="on"/><meta property="fb:app_id" content="458584288257241"/><meta name="theme-color" content="#400090"/><meta name="viewport" content="width=device-width, initial-scale=1"/><meta name="description" lang="en-US" content="Find the latest Altisource Asset Management Corporation (AAMC) stock quote, history, news and other vital information to

,Ticker,Market Cap,Earnings Date Min,Earnings Date Max
0,A,35.215B,"Aug 15, 2023","Aug 15, 2023"
1,AA,5.111B,"Oct 17, 2023","Oct 23, 2023"
2,AAA,N/A,N/A,N/A
3,AAAU,N/A,N/A,N/A
4,AAC,N/A,N/A,N/A
5,AACG,38.535M,"Aug 10, 2023","Aug 10, 2023"
6,AACI,99.466M,N/A,N/A
7,AACT,N/A,N/A,N/A
8,AADI,141.246M,"Nov 07, 2023","Nov 13, 2023"
9,AADR,N/A,N/A,N/A


In [20]:
# Connect to SQLite database (will be created if doesn't exist)
conn = sq.connect('../market_data.db')

# Insert data into a table called "stock_data"
df.to_sql('mkt_cap_next_earnings', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()